In [32]:
import re
import sqlite3
from urllib.request import urlopen
from html import unescape

In [33]:
def main() :
    html = fetch('http://www.hanbit.co.kr/store/books/full_book_list.html')
    books = scrape(html)
    save('books.db', books)
    
    with open('full_book_list.html', 'w') as fw:
        fw.write(html)

In [34]:
def fetch(url):
    f = urlopen(url)
    encoding = f.info().get_content_charset(failobj="utf-8")
    html = f.read().decode(encoding)
    return html

In [35]:
def scrape(html) :
    books = []
    for partial_html in re.findall(r'<td class ="left"><a.*?</td>', html, re.DOTALL):
        url = re.search(r'<a href="(.*?)">', partial_html).group(1)
        url = 'http://www.hanbit.co.kr' + url
        
        title = re.sub(r'<.*?>', '', partial_html)
        title = unescape(title)
        books.append({'url' : url, 'title' : title})
    return books

In [36]:
def save(db_path, books) :
    conn = sqlite3.connect(db_path)
    c = conn.cursor()
    c.execute('DROP TABLE IF EXISTS books')
    c.execute('''
        CREATE TABLE books (
            title text,
            url text
        )
    ''')
    
    c.executemany('INSERT INTO books VALUES (:title, :url)', books)
    conn.commit()
    conn.close()

In [37]:
if __name__ == '__main__' :
    main()

In [38]:
with sqlite3.connect('books.db') as conn :
    c = conn.cursor()
    c.execute('SELECT * FROM books')
    for row in c.fetchall():
        print(row)

In [39]:
!pip install lxml
!pip install cssselect

In [40]:
import lxml.html

In [43]:
tree = lxml.html.parse('full_book_list.hmtl')
html = tree.getroot()

OSError: Error reading file 'user/full_book_list.hmtl': failed to load external entity "user/full_book_list.hmtl"

In [ ]:
for a in html.cssselect('a'):
    print(a.get('href'), a.text)

In [53]:
!pip install beautifulsoup4

In [54]:
from bs4 import BeautifulSoup

In [21]:
with open('full_book_list.html') as f:
    soup = BeautifulSoup(f, 'html.parser')

NameError: name 'BeautifulSoup' is not defined

In [74]:
for a in soup.find_all('a'):
    print(a.get('href'), a.text)

NameError: name 'soup' is not defined